In [1]:
import pandas as pd
import networkx as nx

import xml.etree.ElementTree as ET

from itertools import combinations

In [2]:
pathway = "hsa00510"

tree = ET.parse("../data/human_KGML/{}.xml".format(pathway))
root = tree.getroot()

node_elements  = [elem for elem in root.findall("entry") if elem.get("type") != "group"] # alternatively `root.findall("./entry[@type='group']")`
group_elements = [elem for elem in root.findall("entry") if elem.get("type") == "group"]
edge_elements  = root.findall("relation")

In [3]:
root.attrib

{'image': 'http://www.kegg.jp/kegg/pathway/hsa/hsa00510.png',
 'link': 'http://www.kegg.jp/kegg-bin/show_pathway?hsa00510',
 'name': 'path:hsa00510',
 'number': '00510',
 'org': 'hsa',
 'title': 'N-Glycan biosynthesis'}

In [4]:
def get_node_attributes_from_entry(entry): 
    
    node_attributes = {
        "type": entry.get("type"), # i.e. 'gene'
        "id": int(entry.get("id")) # i.e. 84
    }
    
    # If compound node (protein complexes)
    if node_attributes["type"] == "group": 
        node_attributes["components"] = [int(x.get("id")) for x in entry.findall("component")]
        return node_attributes 
    else: 
        node_attributes["aliases"] = entry[0].get("name") # i.e. 'ALDOA, ALDA, GSD12, HEL-S-87p...'
        node_attributes["hsa_tags"] = entry.get("name") # i.e. 'hsa:226 hsa:229 hsa:230'
        node_attributes["first_name"] = node_attributes["aliases"].split(", ")[0].split("...")[0] # i.e. 'ALDOA'
    
    return node_attributes


def get_edge_attributes_from_entry(entry):
    
    edge_attributes = {
        "node1": int(entry.get("entry1")),
        "node2": int(entry.get("entry2")),
        "type": entry.get("type")
    }
    
    for attribute in entry: 
        
        interaction = attribute.get("name")
        
        if interaction in ["activation", "expression", "indirect effect"]: 
            edge_attributes["sign"] = 1
        elif interaction in ["inhibition", "repression"]: 
            edge_attributes["sign"] = -1
        elif interaction in ["binding/association"]: 
            edge_attributes["sign"] = 0
            
        elif interaction == "phosphorylation": 
            edge_attributes["phosphorylation"] = 1
        elif interaction == "dephosphorylation": 
            edge_attributes["phosphorylation"] = -1
            
        elif interaction == "glycosylation":
            edge_attributes["glycosylation"] = 1
            
        elif interaction == "ubiquitination": 
            edge_attributes["ubiquitination"] = 1
            
        elif interaction == "methylation": 
            edge_attributes["methylation"] = 1
        
        # Force edge through compound node. Unfortunately, these interactions are poorly annotated in KGML.
        # These should be of type "ECrel", but may be mistakenly annotated as "PPrel" in KGMLs.
        elif interaction == "compound": 
            edge1 = {"node1": edge_attributes["node1"], "node2": int(attribute.get("value")), "type": "ECrel"}
            edge2 = {"node2": edge_attributes["node2"], "node2": int(attribute.get("value")), "type": "ECrel"}
        
        else: 
            print(interaction)
            
        # Fill in sign for implicit interactions
        if len(set(edge_attributes.keys()) & set(["phosphorylation", "glycosylation", "ubiquitination", "methylation"])) > 0:
            edge_attributes["sign"] = 1
            
    return edge_attributes


def replace_group_edges(edge_attributes_df, group_obj):
    
    group_id = group_obj["id"] 
    group_members = group_obj["components"]
    n_members = len(group_members)
    initial_length = len(edge_attributes_df)
    
    # Split into two dataframes based on `node1` column
    edges_without_df, edges_with_df = [x for _, x in edge_attributes_df.groupby(edge_attributes_df['node1'] == group_id)]
    # Duplicate rows where `node1` contains the `group_id`
    expanded_edges_df = pd.concat([edges_with_df]*n_members).sort_index() 
    # Replace `node` column with repeating list of `group_members`
    expanded_edges_df["node1"] = group_members*len(edges_with_df)
    # Concatenate the new dataframes and reset index
    edge_attributes_df = pd.concat([expanded_edges_df, edges_without_df]).reset_index(drop=True)
    
    # Do the same procedure with `node2` column
    edges_without_df, edges_with_df = [x for _, x in edge_attributes_df.groupby(edge_attributes_df['node2'] == group_id)]
    expanded_edges_df = pd.concat([edges_with_df]*n_members).sort_index() 
    expanded_edges_df["node2"] = group_members*len(edges_with_df)
    edge_attributes_df = pd.concat([expanded_edges_df, edges_without_df]).reset_index(drop=True)
    
    # Add edges between group members. `sign` will implicity be assigned 0 to indicate no directionality. 
    group_rows = [{"node1": a, "node2": b, "type":"complex"} for a,b in combinations(group_members, 2)]
    edge_attributes_df = edge_attributes_df.append(group_rows, ignore_index=True).fillna(0)
    
    print("{} members in group {}. {} edges added.".format(n_members, group_id, len(edge_attributes_df)-initial_length))
        
    return edge_attributes_df

In [5]:
node_attributes = [get_node_attributes_from_entry(element) for element in node_elements]
node_attributes_df = pd.DataFrame(node_attributes).set_index("id")

In [6]:
node_attributes_df.head()

,aliases,first_name,hsa_tags,type
id,,,,
38,"ALG14, CMS15",ALG14,hsa:199857,gene
39,"DPM3, CDG1O",DPM3,hsa:54344,gene
40,"DPM2, CDG1U",DPM2,hsa:8818,gene
41,Various types of N-glycan biosynthesis,Various types of N-glycan biosynthesis,path:map00513,map
42,G10596,G10596,gl:G10596,compound


In [7]:
group_attributes = [get_node_attributes_from_entry(element) for element in group_elements]
group_attributes

[{'components': [100, 39, 40], 'id': 171, 'type': 'group'},
 {'components': [158, 94], 'id': 172, 'type': 'group'},
 {'components': [38, 99], 'id': 173, 'type': 'group'}]

In [10]:
edge_attributes_df = pd.DataFrame([get_edge_attributes_from_entry(x) for x in edge_elements if x.get("type") in ["PPrel", "CRel"]])
edge_attributes_df.head()

""


In [9]:
# Replace group nodes
for group_attribute in group_attributes: 
    edge_attributes_df = replace_group_edges(edge_attributes_df, group_attribute)

KeyError: 'node1'

In [12]:
# Create directed networkx graph
G_dir = nx.DiGraph()
G_dir.add_nodes_from([(key, dic) for key, dic in node_attributes_df.to_dict("index").items()])
G_dir.add_edges_from([(row["node1"], row["node2"], row[edge_attributes_df.columns[2:]].to_dict()) for _,row in edge_attributes_df.iterrows()])
nx.relabel_nodes(G_dir, {node_id:G_dir.node[node_id]["first_name"] for node_id in G_dir.nodes()}, copy=False)

# Create undirected networkx graph. Useful for finding interactions with ambiguous direction
G_und = nx.Graph()
G_und.add_nodes_from([(key, dic) for key, dic in node_attributes_df.to_dict("index").items()])
G_und.add_edges_from([(row["node1"], row["node2"], row[edge_attributes_df.columns[2:]].to_dict()) for _,row in edge_attributes_df.iterrows()])
nx.relabel_nodes(G_und, {node_id:G_und.node[node_id]["first_name"] for node_id in G_und.nodes()}, copy=False)

In [18]:
def get_KEGG_interactors(protein, G_directed, G_undirected):
    
    if protein not in G_directed: return {}
    
    direct_interactions = G_directed[protein]
    ambiguous_interactions = {node: attr for node, attr in G_undirected[protein].items() if attr["sign"] == 0}
    
    return {**direct_interactions, **ambiguous_interactions}

In [20]:
get_KEGG_interactors("PPP1R12A", G_dir, G_und)

{'MYL12B': {'phosphorylation': -1.0, 'sign': 1.0, 'type': 'PPrel'}}